In [1]:
import json
import pandas as pd
import numpy as np
import os

from sqlalchemy.engine import create_engine

from minio import Minio
from minio.error import S3Error

In [2]:
# COS_API_KEY_ID = os.environ.get("COS_API_KEY_ID", "none")
COS_ENDPOINT = os.environ.get("COS_ENDPOINT", "ibm-lh-lakehouse-minio-svc.cpd.svc.cluster.local:9000")
COS_ACCESS_KEY = os.environ.get("COS_ACCESS_KEY", "none")
COS_SECRET_KEY = os.environ.get("COS_SECRET_KEY", "none")



# print("key: " + COS_API_KEY_ID)
# if COS_API_KEY_ID == "none":
#     print("No COS API KEY!!")
#     #for name, value in os.environ.items():
#     #    print("{0}: {1}".format(name, value))

In [3]:
# ibm s3
# import ibm_boto3
# from ibm_botocore.client import Config, ClientError

In [25]:
# Parameters

description="Return all the ESG dimensions for a single portfolio ticker"

output="output.json"

input={"ticker": "AAA"}

results={"Ticker": "ABNP",
         "ISIN": "OTOY0BBKWAA1",
         "Industry": "Homebuilding and construction supplies",
         "ESG_properties": [
             {"C02": 0.59},
             {"TotalWaste": 0.86}, 
             {"EnvProducts": None}, 
             {"EnvRND": 0.87}, 
             {"WaterUse": 0.77}, 
             {"EnergyUse": 0.61}, 
             {"PolicyHumanRights": 0.84}, 
             {"PolicyResponsibleMarketing": None}, 
             {"ProductQualityMonitoring": None}, 
             {"PolicyDataPrivacy": 0.45}, 
             {"WomenEmployees": 0.33}, 
             {"AvgTrainingHours": 0.97}, 
             {"TradeUnionRep": 0.48}, 
             {"LostDays": 0.51}]}

Test curl post:

    curl -X POST https://datamon.15nwk2ygtk89.eu-de.codeengine.appdomain.cloud/api/ESGFastTickerService -H 'Content-Type: application/json' -d '{"portfolio": "ABNF"}'

In [26]:
ticker_param = input['ticker']
ticker_param

'AAA'

In [27]:
# read data from presto engine
LH_ENDPOINT = os.environ.get("LH_ENDPOINT", "ibm-lh-lakehouse-presto-01-presto-svc.cp4d.svc.cluster.local:8443")
LH_USER = os.environ.get("LH_USER", "none")
LH_PASSWORD = os.environ.get("LH_PASSWORD", "none")
LH_CATALOG = os.environ.get("LH_CATALOG", "delivery")
LH_SCHEMA = os.environ.get("LH_SCHEMA", "provider_a")
LH_PROTOCOL = os.environ.get("LH_PROTOCOL", "https")

url = "presto://" + LH_USER + ":" + LH_PASSWORD + "@" + LH_ENDPOINT + "/" + LH_CATALOG + "/" + LH_SCHEMA
url 
engine = create_engine(url, connect_args={'protocol': LH_PROTOCOL})
conn = engine.connect()

select_stmt = """
SELECT
  *
FROM
  "esg_consolidated"
 WHERE
  "Ticker" = %s
"""
%time source = pd.read_sql_query(select_stmt, con=engine, params=[ticker_param])



CPU times: user 333 ms, sys: 34.5 ms, total: 368 ms
Wall time: 3.04 s


In [28]:
source.head(5)

,Ticker,Isin,Industry,Co2,Totalwaste,Envproducts,Envrnd,Wateruse,Energyuse,Policyhumanrights,Policyresponsiblemarketing,Productqualitymonitoring,Policydataprivacy,Womenemployees,Avgtraininghours,Tradeunionrep,Lostdays
0,AAA,XXXXXXXXXX,Uranium,0.72,0.69,0.56,0.09,0.98,0.21,0.9,NaN,0.53,0.99,0.06,0,0.92,0.95
1,AAA,XXXXXXXXXX,Oil and gas related equipment and services,NaN,0.58,0.48,0.91,0.11,NaN,NaN,0.77,0.08,0.39,0.89,0,0.74,0.39


In [29]:
# Constants for IBM COS values
# COS_ENDPOINT = "https://s3.us-east.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
#COS_API_KEY_ID = # <api-key> eg "W00YixxxxxxxxxxMB-odB-2ySfTrFBIQQWanc--P3byk"
# COS_INSTANCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/b71ac2564ef0b98f1032d189795994dc:f7e4281d-a7f2-4923-8e6b-07f99ac93bbd::" # <service-instance-id> eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003xxxxxxxxxx1c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"

# Create client 
# s3_client = ibm_boto3.client("s3",
#     ibm_api_key_id=COS_API_KEY_ID,
#     ibm_service_instance_id=COS_INSTANCE_CRN,
#     config=Config(signature_version="oauth"),
#     endpoint_url=COS_ENDPOINT
# )

# COS_ENDPOINT='ibm-lh-lakehouse-minio-svc.cpd.svc.cluster.local:9000'

# minio_client = Minio(
#     endpoint=COS_ENDPOINT,
#     secure=False,
#     access_key=COS_ACCESS_KEY,
#     secret_key=COS_SECRET_KEY
# )


# file_name = "provider_a/ESG-consolidated.csv"
# bucket = "delivery"




# print("Retrieving item from bucket: {0}, key: {1}".format(bucket, file_name))
# try:
#     minio_client.fget_object(bucket, file_name,file_name)
# except S3Error as e:
#     e


In [30]:
# source = pd.read_csv(file_name)
# source.head(5)

In [31]:
# source = source[source['Ticker'] == input['ticker']]
# source

In [32]:
output_dict={}
output_dict['Ticker'] = source['Ticker'].values[0]
output_dict['ISIN'] = source['Isin'].values[0]
output_dict['Industry'] = source['Industry'].values[0]
output_dict

{'Ticker': 'AAA', 'ISIN': 'XXXXXXXXXX', 'Industry': 'Uranium'}

In [33]:
cols = source.columns.values
cols = cols[3:]
cols

array(['Co2', 'Totalwaste', 'Envproducts', 'Envrnd', 'Wateruse',
       'Energyuse', 'Policyhumanrights', 'Policyresponsiblemarketing',
       'Productqualitymonitoring', 'Policydataprivacy', 'Womenemployees',
       'Avgtraininghours', 'Tradeunionrep', 'Lostdays'], dtype=object)

In [34]:
property_list=[]
for dimension in cols:
    obj_dict={}
    obj_dict[dimension] = source[dimension].values[0]
    property_list.append(obj_dict)
property_list

[{'Co2': 0.72},
 {'Totalwaste': 0.69},
 {'Envproducts': 0.56},
 {'Envrnd': 0.09},
 {'Wateruse': 0.98},
 {'Energyuse': 0.21},
 {'Policyhumanrights': 0.9},
 {'Policyresponsiblemarketing': nan},
 {'Productqualitymonitoring': 0.53},
 {'Policydataprivacy': 0.99},
 {'Womenemployees': 0.06},
 {'Avgtraininghours': 0},
 {'Tradeunionrep': 0.92},
 {'Lostdays': 0.95}]

In [35]:
output_dict["ESG_properties"] = property_list
output_dict

{'Ticker': 'AAA',
 'ISIN': 'XXXXXXXXXX',
 'Industry': 'Uranium',
 'ESG_properties': [{'Co2': 0.72},
  {'Totalwaste': 0.69},
  {'Envproducts': 0.56},
  {'Envrnd': 0.09},
  {'Wateruse': 0.98},
  {'Energyuse': 0.21},
  {'Policyhumanrights': 0.9},
  {'Policyresponsiblemarketing': nan},
  {'Productqualitymonitoring': 0.53},
  {'Policydataprivacy': 0.99},
  {'Womenemployees': 0.06},
  {'Avgtraininghours': 0},
  {'Tradeunionrep': 0.92},
  {'Lostdays': 0.95}]}

In [36]:
jsonString = json.dumps(output_dict, default=str)
jsonString

'{"Ticker": "AAA", "ISIN": "XXXXXXXXXX", "Industry": "Uranium", "ESG_properties": [{"Co2": 0.72}, {"Totalwaste": 0.69}, {"Envproducts": 0.56}, {"Envrnd": 0.09}, {"Wateruse": 0.98}, {"Energyuse": 0.21}, {"Policyhumanrights": 0.9}, {"Policyresponsiblemarketing": NaN}, {"Productqualitymonitoring": 0.53}, {"Policydataprivacy": 0.99}, {"Womenemployees": 0.06}, {"Avgtraininghours": "0"}, {"Tradeunionrep": 0.92}, {"Lostdays": 0.95}]}'

In [37]:
 # Writing to sample.json
with open(output, "w") as outfile:
    outfile.write(jsonString)